In [1]:
%pylab inline
from pulp import *
import pandas as pd
import os

Populating the interactive namespace from numpy and matplotlib


### Read data and set user-defined parameters
Only update data, change buses/lines setting, change fuel price/emission standard in this section

In [48]:
# Generator data (update in raw excel file with same format)
raw_data=pd.read_excel('/raw.xlsx',sheet_name='Sheet2', index_col=0)

# Load data in different time and season need to be dispatched
load_data=pd.read_excel('/loaddata.xlsx')

In [49]:
# required model input (can have different generator list, bus number and lines)
Gen_list=list(raw_data.index)
Bus_list=['1','2','3']
Line_list=[('1','2'),('2','3'),('1','3')]

load_dict={'1': 0.15, '2': 0.5, '3': 0.35}

fuel_price={'NG':3.06, 'Coal': 2.16, 'Oil': 9.83} # $/mmBtu
SO2_stan=0.7 # lbs/mmBtu
NOx_stan=0.15 # lbs/mmBtu
GHG_stan=600 # lbs/MWh
carbon_price=0 # $/ton

## parameters of transmission lines
line_capacity_dict = dict(zip(Line_list, [1100, 2000, 2000]))
line_impedance = dict(zip(Line_list, [0.2,0.2,-0.2]))# here assume to all be 0.2/unit

### Pre-process data and make dictionaries

In [50]:
# first calculate MC of each generator
raw_data['fuel_cost']=raw_data.apply(lambda x: fuel_price[x['Fuel']], axis=1) #$/mmbtu
raw_data['carbon_cost']=raw_data.apply(lambda x: x['CO2']* carbon_price/2205, axis=1 ) # 1 metric ton CO2 = 2205 lb, gives $/MWh
raw_data['MC']=raw_data.apply(lambda x: x['fuel_cost']*x['Heat Rate']/1000 + x['carbon_cost'], axis=1) # $/MWh

In [54]:
# Build dictionaries

## dictionaries about each bus
bus1_gen=list((raw_data.loc[raw_data['Bus']==1]).index)
bus2_gen=list((raw_data.loc[raw_data['Bus']==2]).index)
bus3_gen=list((raw_data.loc[raw_data['Bus']==3]).index)
bus_dict={'1': bus1_gen, '2': bus2_gen, '3': bus3_gen}

## dictionaries about generators
mc_dict=(raw_data.loc[:,'MC']).to_dict() # $/MWh
fuel_dict=(raw_data.loc[:,'Fuel']).to_dict()
maxgen_dict=(raw_data.loc[:,'capacity(MW)']).to_dict() # MW
mingen_dict=(raw_data.loc[:,'Min Generation']).to_dict() # MW
HR_dict=(raw_data.loc[:,'Heat Rate']).to_dict() # Btu/kWh
NOx_dict=(raw_data.loc[:,'Nox']).to_dict() # lb/MWh
SO2_dict=(raw_data.loc[:,'SO2']).to_dict() # lb/MWh
CO2_dict=(raw_data.loc[:,'CO2']).to_dict() # lb/MWh
CH4_dict=(raw_data.loc[:,'CH4']).to_dict() # lb/MWh
N2O_dict=(raw_data.loc[:,'N2O']).to_dict() # lb/MWh

### Solve and collect results

In [55]:
# this is the core function to solve the constrained dispatch problem
# load: MW; inclde_emission_cst = False during early planning
def solve(Load,include_emission_cst): 
    # Define optimization problem
    prob = LpProblem('Constrained UCED', LpMinimize)
    
    # Decision variables
    gens= LpVariable.dicts('generation',Gen_list)
    flows= LpVariable.dicts('flow', Line_list)

    # Add Constraints
    ## Supply meets demand
    prob += Load - lpSum(gens[g] for g in Gen_list) >= 0, 'Supply meet demand'
    ## Generator constraints
    for g in Gen_list:
        prob += gens[g] <= maxgen_dict[g], 'Max generation for '+ g
        prob += gens[g] >= mingen_dict[g], 'Min generation for ' + g  
    ## Transmission line constraints:
    for l in Line_list:
        prob += flows[l] <= line_capacity_dict[l], 'Max transmission capacity for line ' + l[0] + l[1]
        prob += flows[l] >= -line_capacity_dict[l], 'Inverse max transmission capacity for line ' + l[0] + l[1] 
    ## Bus balance
    for b in Bus_list:
        prob += lpSum(gens[g] for g in bus_dict[b]) - lpSum(v for k,v in flows.items() if k[0]==b) + lpSum(v for k,v in flows.items() if k[1]==b) >= Load * load_dict[b] , 'Bus balance for '+ b
    ## KVL
    prob += lpSum(flows[l]*line_impedance[l] for l in Line_list) == 0, 'KVL along the loop'    
    ## hourly emissions
    if include_emission_cst:
        print('SO2, NOx and GHG emission standard constraints are included')
        prob += lpSum(gens[g] * 1 * SO2_dict[g] for g in Gen_list if fuel_dict[g]=='Coal') <= SO2_stan * lpSum(gens[g] * HR_dict[g] / 1000 for g in Gen_list if fuel_dict[g]=='Coal'), 'SO2 emission constraint'
        prob += lpSum(gens[g] * 1 * NOx_dict[g] for g in Gen_list if fuel_dict[g]=='Coal') <= NOx_stan * lpSum(gens[g] * HR_dict[g] / 1000 for g in Gen_list if fuel_dict[g]=='Coal'), 'NOx emission constraint'
        prob += lpSum(gens[g] * 1 * (CO2_dict[g] + CH4_dict[g] + N2O_dict[g]) for g in Gen_list) <= GHG_stan * lpSum(gens[g] for g in Gen_list), 'GHG emission constraint'

    # Objective function
    prob += lpSum(gens[g] * mc_dict[g] for g in Gen_list)
    
    # Other kpis
    coal_CO2 = LpAffineExpression([(gens[g],CO2_dict[g]) for g in Gen_list if fuel_dict[g]=='Coal'])
    NG_CO2 = LpAffineExpression([(gens[g],CO2_dict[g]) for g in Gen_list if fuel_dict[g]=='NG'])
    Oil_CO2 = LpAffineExpression([(gens[g],CO2_dict[g]) for g in Gen_list if fuel_dict[g]=='Oil'])
    
    coal_consumption = LpAffineExpression([(gens[g], HR_dict[g] / 1000) for g in Gen_list if fuel_dict[g]=='Coal'])
    NG_consumption = LpAffineExpression([(gens[g] , HR_dict[g] / 1000) for g in Gen_list if fuel_dict[g]=='NG'])
    Oil_consumption = LpAffineExpression([(gens[g] , HR_dict[g] / 1000) for g in Gen_list if fuel_dict[g]=='Oil'])
    
    CO2_emission = LpAffineExpression([(gens[g] , CO2_dict[g]) for g in Gen_list])    
    
    # solve results
    prob.solve()
    print("The dispatch solution of ", Load, "is: ", LpStatus[prob.status])
    total_cost=value(prob.objective)
    generation_schedule=[]
    for v in prob.variables():
        if v.name.find("flow")!= 0:
            generation_schedule.append([v.name.split('_')[1], v.varValue])
        else:
            generation_schedule.append([v.name.split('flow_')[1], v.varValue])
    shp = pd.DataFrame([{'Constraint_name':name, 'shadow_price':c.pi, 'slack':c.slack}
                for name, c in prob.constraints.items()])
    kpis={'Load value': Load, 'Total cost($)':total_cost, 'Coal_co2': value(coal_CO2),'NG_co2': value(NG_CO2), 'Oil_co2': value(Oil_CO2),
    'Coal_consum (mmbtu)': value(coal_consumption),'NG_consum (mmbtu)': value(NG_consumption), 'Oil_consum (mmbtu)': value(Oil_consumption),
    'CO2_emission (lb)': value(CO2_emission)}
    solution_kpi=pd.DataFrame(kpis, index=[0])
    
    return (generation_schedule, shp, solution_kpi)

In [56]:
# collect results
df_kpi=pd.DataFrame(columns=('Load value','Total cost($)','Coal_co2','NG_co2','Oil_co2','Coal_consum (mmbtu)','NG_consum (mmbtu)','Oil_consum (mmbtu)','CO2_emission (lb)'))
writer_gs=pd.ExcelWriter('/generation_schedule_sort.xlsx',engine='openpyxl')
writer_shp=pd.ExcelWriter('/shadow_price.xlsx',engine='openpyxl')
custom_order=Gen_list.copy()
custom_order.extend(["('1',_'2')", "('1',_'3')", "('2',_'3')"])

for loads in load_data['Load value'].values:
    generation_schedule, shp, solution_kpi=solve(loads,include_emission_cst=False)
    df_kpi=pd.concat([df_kpi, solution_kpi])
    df_gs=pd.DataFrame(generation_schedule, columns=['Variables','Value'])
    df_gs['Variables']=df_gs['Variables'].astype('category')
    df_gs['Variables'].cat.reorder_categories(custom_order, inplace=True)
    df_gs.sort_values('Variables', inplace=True)
    df_gs.to_excel(excel_writer=writer_gs, sheet_name=str(loads))
    shp.to_excel(excel_writer=writer_shp, sheet_name=str(loads))
    writer_gs.save()
    writer_shp.save()

all_load_data=pd.merge(load_data, df_kpi, on=(['Load value']), how='outer')
all_load_data.to_excel('/kpis.xlsx')

The dispatch solution of  4198 is:  Optimal
The dispatch solution of  4765 is:  Optimal
The dispatch solution of  5974 is:  Optimal
The dispatch solution of  4359 is:  Optimal
The dispatch solution of  3726 is:  Optimal
The dispatch solution of  4050 is:  Optimal
The dispatch solution of  5009 is:  Optimal
The dispatch solution of  3780 is:  Optimal
The dispatch solution of  6189 is:  Optimal
The dispatch solution of  6951 is:  Optimal
The dispatch solution of  7618 is:  Optimal
The dispatch solution of  6653 is:  Optimal
The dispatch solution of  5749 is:  Optimal
The dispatch solution of  6501 is:  Optimal
The dispatch solution of  7338 is:  Optimal
The dispatch solution of  6389 is:  Optimal
